## Яндекс Практикум, курс "Инженер Машинного Обучения" (2024 г.)
## Проект 4-го спринта: "Создание рекомендательной системы"

## Этап 4. Сервис рекомендаций

__Постановка задачи__

Напишите FastAPI-микросервис для выдачи рекомендаций, который:
- принимает запрос с идентификатором пользователя и выдаёт рекомендации,
- учитывает историю пользователя,
- смешивает онлайн- и оффлайн-рекомендации.

В README опишите стратегию смешивания онлайн- и оффлайн-рекомендаций.

Выполните тестирование микросервиса в ноутбуке `part_3_test.ipynb`:
- для пользователя без персональных рекомендаций,
- для пользователя с персональными рекомендациями, но без онлайн-истории,
- для пользователя с персональными рекомендациями и онлайн-историей.

### Инициализация

In [1]:
import numpy as np
import pandas as pd
import requests

In [2]:
import sys

# Проверяем, в каком окружении работаем
print(sys.executable)

/home/mle-user/.venv/bin/python


In [3]:
# Адреса сервисов
recommendations_url = "http://127.0.0.1:8000"
features_store_url = "http://127.0.0.1:8010"
events_store_url = "http://127.0.0.1:8020"

# Общий заголовок для всех запросов
headers = {"Content-type": "application/json", "Accept": "text/plain"}

Для запуска сервисов с помощью uvicorn необходимо выполнить 3 команды в 3-х разных терминалах, находясь в папке проекта:

```
uvicorn recommendations_service:app
uvicorn events_service:app --port 8020
uvicorn features_service:app --port 8010
```

### Загрузка данных

Загрузким треки для отображения рекомендаций со всеми полями, а также сами рекомендации, чтобы выбирать из них пользователей и объекты для тестирования

In [4]:
items = pd.read_parquet("items.parquet")
recommendations = pd.read_parquet('recommendations.parquet')
similar = pd.read_parquet('similar.parquet')

### Получение рекомендаций по умолчанию

Напишем вспомогательную функцию для отображения объектов по их идентификаторам

In [5]:
def display_items(item_ids):

    item_columns_to_use = ["item_id", "name", "genres", "albums", "artists"]
    
    items_selected = items.query("item_id in @item_ids")[item_columns_to_use]
    items_selected = items_selected.set_index("item_id").reindex(item_ids)
    items_selected = items_selected.reset_index()
    
    display(items_selected)

Отправляем запрос для получения рекомендаций по умолчанию, т.е. из числа топ-треков

In [6]:
params = {'k': 10}
resp = requests.post(recommendations_url + "/recommendations_default", headers=headers, params=params)
if resp.status_code == 200:
    resp = resp.json()
    display_items(resp['recs'])
else:
    print(f"status code: {resp.status_code}")

,item_id,name,genres,albums,artists
0,53404,Smells Like Teen Spirit,"[alternative, allrock, rock]",[Smells Like Teen Spirit / In Bloom / On A Pla...,[Nirvana]
1,178529,Numb,"[numetal, metal]","[Meteora, 00s Rock Anthems]",[Linkin Park]
2,37384,Zombie,"[allrock, rock]","[90s Alternative, MNM Sing Your Song: Back To ...",[The Cranberries]
3,6705392,Seven Nation Army,[alternative],"[Pay Close Attention : XL Recordings, Radio 1 ...",[The White Stripes]
4,33311009,Believer,"[allrock, rock]","[Horoscope Tunes: Cancer, Workout Music Hits 2...",[Imagine Dragons]
5,328683,Bring Me To Life,"[alternative, allrock, rock]","[Mental Health 2020, Monster Millennium Hits, ...",[Evanescence]
6,148345,Californication,"[allrock, rock]","[The Studio Album Collection 1991-2011, Greate...",[Red Hot Chili Peppers]
7,795836,Shape Of My Heart,"[allrock, pop, rock]","[25 Years, Сентиментальные рок-песни, Ten Summ...",[Sting]
8,630670,"You're Gonna Go Far, Kid",[punk],"[You're Gonna Go Far, Kid, Sleep Music Rock Ed...",[The Offspring]
9,178495,In the End,"[numetal, metal]","[Antyradio: Najlepszy Rock Na Swiecie Vol. 4, ...",[Linkin Park]


### Получение персональных рекомендаций без онлайн-истории

In [7]:
# Выберем произвольного пользователя случайным образом
user_id = recommendations['user_id'].sample().iat[0]
user_id

12081

In [9]:
# Посмотрим, какие есть для него рекомендации
recommendations.query('user_id == @user_id')

,user_id,item_id,als_score,cnt_score,tracks_played_by_user,score,rank
6,12081,73495,0.812035,0.927586,2009,0.997894,1


Сравним содержимое файла с результатом запроса

In [16]:
# Отправляем запрос для получения только оффлайн-рекомендаций
params = {'user_id': user_id, 'k': 10}
resp = requests.post(recommendations_url + "/recommendations_offline", headers=headers, params=params)
if resp.status_code == 200:
    resp = resp.json()
    display_items(resp['recs'])
else:
    recs = []
    print(f"status code: {resp.status_code}")

,item_id,name,genres,albums,artists
0,73495,Angels (Love Is The Answer),"[dance, electronics, pop]",[Valentine's Day 2022 - cele mai frumoase melo...,[Morandi]


### Получение персональных рекомендаций с онлайн-историей

Выберем 3 произвольных трека, имеющих i2i-рекомендации, и добавим их в онлайн-историю пользователя

In [11]:
# Выбираем 3 объекта
item_ids = similar['item_id_1'].unique()[:3]
print(item_ids)

# Добавляем в онлайн-историю пользователя
for item_id in item_ids:
    params = {"user_id": user_id, "item_id": item_id}
    resp = requests.post(events_store_url + "/put", headers=headers, params=params)
    if resp.status_code == 200:
        resp = resp.json()
        print(resp['result'])    
    else:
        print(f"status code: {resp.status_code}") 

[549 553 558]
ok
ok
ok


Проверим, что события добавились, причем в обратном порядке

In [12]:
params = {"user_id": user_id, "k": 3}
resp = requests.post(events_store_url + "/get", headers=headers, params=params)
if resp.status_code == 200:
    resp = resp.json()
    print(resp['events'])    
else:
    print(f"status code: {resp.status_code}")

[558, 553, 549]


Протестируем работу сервиса `feautures_service`. 

Возмьмем последний объект из списка выше и найдем похожие на него треки из файла `similar.parquet`

In [13]:
item_id_1 = item_ids[-1]
similar.query('item_id_1 == @item_id_1')

,score,item_id_1,item_id_2
23,0.924216,558,1706463
24,0.908504,558,1706462
25,0.904736,558,550048
26,0.897181,558,69658
27,0.894441,558,553
28,0.886535,558,71239
29,0.885370,558,2277920
30,0.881694,558,71965
31,0.881250,558,34300
32,0.873259,558,10260869


Теперь получим тот же набор объектов с помощью сервиса `features_service`

In [17]:
params = {"item_id": item_id_1, "k": 10}
resp = requests.post(features_store_url +"/similar_items", headers=headers, params=params)
if resp.status_code == 200:
    resp = resp.json()
    print(resp['item_id_2'])    
else:
    print(f"status code: {resp.status_code}")    

[1706463, 1706462, 550048, 69658, 553, 71239, 2277920, 71965, 34300, 10260869]


Протестируем запрос к сервису `reccomendations_service` для получения объектов, похожих на 3 последних трека из онлайн-истории данного пользователя

In [18]:
params = {'user_id': user_id, 'k': 10}
resp = requests.post(recommendations_url + "/recommendations_online", headers=headers, params=params)
if resp.status_code == 200:
    resp = resp.json()
    display_items(resp['recs'])
else:
    recs = []
    print(f"status code: {resp.status_code}")

,item_id,name,genres,albums,artists
0,1706462,The Time (Dirty Bit),"[electronics, pop]","[Kids Songs: Game Day!, Party de Revelion 2021...",[Black Eyed Peas]
1,1706463,Don’t Stop The Party,"[electronics, pop]","[The Beginning, NRJ 200% Hits 2011 Vol 2, The ...",[Black Eyed Peas]
2,26771,London Bridge,[rnb],"[Faxina Pop, Летняя дискотека, Ready Steady Ib...",[Fergie]
3,4198,Fergalicious,[rnb],"[Chicks With Kick, The Dutchess Deluxe, Pregam...","[will.i.am, Fergie]"
4,69658,I Got It From My Mama,[electronics],"[Mom Songs, Hip Hop: The Collection 2008, Ulti...",[will.i.am]
5,37402640,Impacto,"[latinfolk, folk]",[El Cartel: The Big Boss],"[Daddy Yankee, Fergie]"
6,631538,Boom Boom Pow,[rnb],"[Now 20, Boom Boom Pow]",[Black Eyed Peas]
7,123846,Lose Control,"[rap, foreignrap]","[Respect M.E., The Cookbook]","[Ciara, Fatman Scoop, Missy Elliott]"
8,12281,My Humps,"[rap, foreignrap]","[Monkey Business, Junior Prom, Horoscope Tunes...",[Black Eyed Peas]
9,78710,Party People,"[rap, rnb, foreignrap]","[The Dutchess Deluxe EP, Brass Knuckles, The D...","[Nelly, Fergie]"


Наконец, получим смешанные рекомендации. На нечетных местах будут оффлайн-рекомендации, на четных - онлайн.

In [19]:
params = {"user_id": user_id, 'k': 10}
resp = requests.post(recommendations_url + "/recommendations", headers=headers, params=params)
if resp.status_code == 200:
    resp = resp.json()
    display_items(resp['recs'])
else:
    print(f"status code: {resp.status_code}")

,item_id,name,genres,albums,artists
0,73495,Angels (Love Is The Answer),"[dance, electronics, pop]",[Valentine's Day 2022 - cele mai frumoase melo...,[Morandi]
1,1706462,The Time (Dirty Bit),"[electronics, pop]","[Kids Songs: Game Day!, Party de Revelion 2021...",[Black Eyed Peas]
2,1706463,Don’t Stop The Party,"[electronics, pop]","[The Beginning, NRJ 200% Hits 2011 Vol 2, The ...",[Black Eyed Peas]
3,26771,London Bridge,[rnb],"[Faxina Pop, Летняя дискотека, Ready Steady Ib...",[Fergie]
4,4198,Fergalicious,[rnb],"[Chicks With Kick, The Dutchess Deluxe, Pregam...","[will.i.am, Fergie]"
5,69658,I Got It From My Mama,[electronics],"[Mom Songs, Hip Hop: The Collection 2008, Ulti...",[will.i.am]
6,37402640,Impacto,"[latinfolk, folk]",[El Cartel: The Big Boss],"[Daddy Yankee, Fergie]"
7,631538,Boom Boom Pow,[rnb],"[Now 20, Boom Boom Pow]",[Black Eyed Peas]
8,123846,Lose Control,"[rap, foreignrap]","[Respect M.E., The Cookbook]","[Ciara, Fatman Scoop, Missy Elliott]"
9,12281,My Humps,"[rap, foreignrap]","[Monkey Business, Junior Prom, Horoscope Tunes...",[Black Eyed Peas]
